In [7]:
import os
import re
import requests
from dotenv import load_dotenv
from datetime import datetime, timedelta
from bs4 import BeautifulSoup

load_dotenv()

apiKey = os.getenv("API_KEY")
flight = os.getenv("FLIGHT")
startAirport = os.getenv("START_AIRPORT")
endAirport = os.getenv("END_AIRPORT")
startDate = os.getenv("START_DATE")

parsedDate = datetime.strptime(
    startDate, "%Y-%m-%dT%H:%M:%S.%f+00:00")
endDate = parsedDate+timedelta(days=1)

flightRegex = re.compile("(?P<airline>\D+)(?P<flightNumber>\d+)")
engines = re.compile("Engines\s+(?P<amount>\d+) x (?P<engines>.+)")

headers = {
    "x-apikey": apiKey,
}


In [8]:
def getFlight():
    m = flightRegex.match(flight)

    payload = {
        "origin": startAirport,
        "destination": endAirport,
        "airline": m.group("airline"),
    }
    url = f"https://aeroapi.flightaware.com/aeroapi/schedules/{parsedDate.strftime('%Y-%m-%d')}/{endDate.strftime('%Y-%m-%d')}"

    print("Sending request...")
    print("URL:", url)
    print("Payload:", payload)
    r = requests.get(url, params=payload, headers=headers)

    print("Response:", r.text)

    return r.json()["scheduled"][0]["fa_flight_id"]


def getEngine(flightId):
    url = f"https://aeroapi.flightaware.com/aeroapi/flights/{flightId}"

    print("Sending request...")
    print("URL:", url)
    r = requests.get(url, headers=headers)

    return r.json()


In [9]:
flightId = getFlight()
engine = getEngine(flightId)

Sending request...
URL: https://aeroapi.flightaware.com/aeroapi/schedules/2022-03-07/2022-03-08
Payload: {'origin': 'ARN', 'destination': 'ALC', 'airline': 'DY'}
Response: {"scheduled":[{"ident":"NOZ5353","ident_icao":"NOZ5353","ident_iata":"DY5353","actual_ident":null,"actual_ident_icao":null,"actual_ident_iata":null,"aircraft_type":"B738","scheduled_in":"2022-03-07T22:45:00Z","scheduled_out":"2022-03-07T18:50:00Z","origin":"ESSA","origin_icao":"ESSA","origin_iata":"ARN","origin_lid":null,"destination":"LEAL","destination_icao":"LEAL","destination_iata":"ALC","destination_lid":null,"fa_flight_id":"NOZ5353-1646446693-airline-0451","meal_service":"Economy: Food for sale","seats_cabin_business":0,"seats_cabin_coach":186,"seats_cabin_first":0}],"links":null,"num_pages":1}
Sending request...
URL: https://aeroapi.flightaware.com/aeroapi/flights/NOZ5353-1646446693-airline-0451


In [10]:
def searchPlane(registration):
    payload = {
        "key": registration,
    }

    header = {
        "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:97.0) Gecko/20100101 Firefox/97.0"
    }

    url = f"https://www.airfleets.net/recherche/"

    print("Sending request...")
    print("URL:", url)
    print("Payload:", payload)
    r = requests.get(url, headers=header, params=payload)

    soup = BeautifulSoup(r.text, "html.parser")
    planeHref = soup.find('main').find_all('table')[2].find_all('tr')[1].find_all('td')[0].find('a')['href']
    print("Plane href:", planeHref)

    return url + planeHref
        
registration = engine['flights'][0]['registration']
planeUrl = searchPlane(registration)
print("Plane URL:", planeUrl)

Sending request...
URL: https://www.airfleets.net/recherche/
Payload: {'key': 'SE-RPI'}
Plane href: ../ficheapp/plane-b737ng-42072.htm
Plane URL: https://www.airfleets.net/recherche/../ficheapp/plane-b737ng-42072.htm


In [16]:
def getEngine(url):
    header = {
        "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:97.0) Gecko/20100101 Firefox/97.0"
    }

    r = requests.get(url, headers=header)

    soup = BeautifulSoup(r.text, "html.parser")

    engine = soup.find('main').find_all('table')[6].find('td').text
    print(engine.strip())
    return engine.strip()


engineSpec = getEngine(planeUrl)



Engines				2 x CFMI CFM56-7B26


In [17]:
engineRegex = re.compile("Engines\s+(?P<amount>\d+) x (?P<engines>.+)")

def parseEngine(engine):
    m = engineRegex.match(engine)
    return [m.group('amount'), m.group('engines')]

parseEngine(engineSpec)


<re.Match object; span=(0, 30), match='Engines\t\t\t\t2 x CFMI CFM56-7B26'>
